In [103]:
import pandas as pd
import networkx as nx
import folium
from geopy.geocoders import Nominatim
from geopy.distance import geodesic


In [104]:
#func: calcular distancia entre coords
def calculate_distance(coord1, coord2):
    return geodesic(coord1, coord2).kilometers

#func: pegar coordenada de cada cidade no mapa
def get_coordinates(city):
    geolocator = Nominatim(user_agent="route_optimizer")
    location = geolocator.geocode(city)
    if location:
        return (location.latitude, location.longitude)
    else:
        return (None, None)

In [105]:
df = pd.read_csv("torch_route.csv")

#classificando pelo inicio da rota
df = df.sort_values(by='date_start')

#pegando coordenadas
df['coordinates'] = df['city'].apply(get_coordinates)

#remover coord erradas
df = df.dropna(subset=['coordinates', 'city'])
df = df[df['coordinates'].apply(lambda x: x[0] is not None and x[1] is not None)]

In [121]:
df.head()

,title,city,date_start,date_end,tag,url,stage_number,coordinates
0,Lighting Ceremony,Olympia,2024-04-15T22:01:00Z,2024-04-16T10:01:00Z,lighting-ceremony,https://olympics.com/en/paris-2024/olympic-tor...,NaN,"(37.638250299999996, 21.630566024011667)"
2,Handover Ceremony,Athens,2024-04-25T22:01:00Z,2024-04-26T17:07:00Z,handover-ceremony,https://olympics.com/en/paris-2024/olympic-tor...,NaN,"(33.9597677, -83.376398)"
3,Crossing the Mediterranean,Marseille,2024-04-26T17:08:00Z,2024-05-07T22:01:00Z,mediterranean-crossing,https://olympics.com/en/paris-2024/olympic-tor...,NaN,"(43.2961743, 5.3699525)"
4,Prologue,Marseille,2024-05-07T22:02:00Z,2024-05-08T20:02:00Z,prologue-marseille,https://olympics.com/en/paris-2024/olympic-tor...,NaN,"(43.2961743, 5.3699525)"
5,Marseille,Marseille,2024-05-08T22:01:00Z,2024-05-09T21:59:00Z,marseille,https://olympics.com/en/paris-2024/olympic-tor...,1.0,"(43.2961743, 5.3699525)"


In [130]:
#criar as linhas do mapa
G = nx.Graph()

#loop por toda a df
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        coord1 = df.iloc[i]['coordinates']
        coord2 = df.iloc[j]['coordinates']
        #calculando distancias
        dist = calculate_distance(coord1, coord2)
        #adicionando marcador em cada loc
        G.add_edge(df.iloc[i]['city'], df.iloc[j]['city'], weight=dist)

In [144]:
df

,title,city,date_start,date_end,tag,url,stage_number,coordinates
0,Lighting Ceremony,Olympia,2024-04-15T22:01:00Z,2024-04-16T10:01:00Z,lighting-ceremony,https://olympics.com/en/paris-2024/olympic-tor...,NaN,"(37.638250299999996, 21.630566024011667)"
2,Handover Ceremony,Athens,2024-04-25T22:01:00Z,2024-04-26T17:07:00Z,handover-ceremony,https://olympics.com/en/paris-2024/olympic-tor...,NaN,"(33.9597677, -83.376398)"
3,Crossing the Mediterranean,Marseille,2024-04-26T17:08:00Z,2024-05-07T22:01:00Z,mediterranean-crossing,https://olympics.com/en/paris-2024/olympic-tor...,NaN,"(43.2961743, 5.3699525)"
4,Prologue,Marseille,2024-05-07T22:02:00Z,2024-05-08T20:02:00Z,prologue-marseille,https://olympics.com/en/paris-2024/olympic-tor...,NaN,"(43.2961743, 5.3699525)"
5,Marseille,Marseille,2024-05-08T22:01:00Z,2024-05-09T21:59:00Z,marseille,https://olympics.com/en/paris-2024/olympic-tor...,1.0,"(43.2961743, 5.3699525)"
...,...,...,...,...,...,...,...,...
68,Essonne,Evry-Courcouronnes,2024-07-21T22:01:00Z,2024-07-22T21:59:00Z,essonne-evry-courcouronnes,https://olympics.com/en/paris-2024/olympic-tor...,64.0,"(48.629966, 2.4381816)"
69,Yvelines,Versailles,2024-07-22T22:01:00Z,2024-07-23T21:59:00Z,yvelines-versailles,https://olympics.com/en/paris-2024/olympic-tor...,65.0,"(48.8035403, 2.1266886)"
70,Hauts-de-Seine,Nanterre > L’Arche de la Défense,2024-07-23T22:01:00Z,2024-07-24T21:59:00Z,hauts-de-seine-nanterre,https://olympics.com/en/paris-2024/olympic-tor...,66.0,"(48.89256445, 2.2358742155740754)"
71,Seine-Saint-Denis,Parc Georges-Valbon,2024-07-24T22:01:00Z,2024-07-25T21:59:00Z,seine-saint-denis-la-courneuve,https://olympics.com/en/paris-2024/olympic-tor...,67.0,"(48.947570150000004, 2.3989473810607276)"


In [125]:
#resolvendo TSP/criando rotas
path = nx.approximation.traveling_salesman_problem(G, cycle=False)

# Criar mapa centrado no inicio da rota

#definindo inicio da rota
start_coords = df[df['city'] == path[0]].iloc[0]['coordinates']
#criando mapa
mymap = folium.Map(location=start_coords, zoom_start=5)

In [126]:
#adicionar marcador no mapa
for i in range(len(path)):
    #criando marcador
    city = path[i]
    coords = df[df['city'] == city].iloc[0]['coordinates']
    folium.Marker(coords, popup=city).add_to(mymap)
    
    #criando linha no mapa
    if i < len(path) - 1:
        next_city = path[i + 1]
        next_coords = df[df['city'] == next_city].iloc[0]['coordinates']
        folium.PolyLine([coords, next_coords], color="blue", weight=1.5, opacity=1).add_to(mymap)

In [127]:
#salvando mapa e mostrando
mymap.save("route_map.html")
mymap
